In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

In [2]:
# ---------------------------
# Paths
# ---------------------------
UNIFIED_DIR = Path("../../data/unified/")
OUT_DIR = Path("../../data/features/")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---------------------------
# Technical indicators function
# ---------------------------
def compute_ema(series, span):
    return series.ewm(span=span, adjust=False).mean()


def compute_rsi(series, window=14):
    delta = series.diff()
    gain = delta.clip(lower=0).rolling(window).mean()
    loss = -delta.clip(upper=0).rolling(window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))


def compute_macd(series):
    ema12 = compute_ema(series, 12)
    ema26 = compute_ema(series, 26)
    return ema12 - ema26

# ---------------------------
# Feature engineering
# ---------------------------
def engineer_features(ticker: str):
    df = pd.read_parquet(UNIFIED_DIR / f"{ticker}.parquet")

    # ---- Technicals ----
    df["ema_34"] = compute_ema(df["close"], 34)
    df["ema_89"] = compute_ema(df["close"], 89)
    df["ema_200"] = compute_ema(df["close"], 200)

    df["rsi_14"] = compute_rsi(df["close"])
    df["macd"] = compute_macd(df["close"])

    df["log_return"] = np.log(df["close"]).diff()
    df["vol_20"] = df["log_return"].rolling(20).std()

    # ---- Lag features ----
    lags = [1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]
    for lag in lags:
        df[f"close_lag_{lag}"] = df["close"].shift(lag)
        df[f"ret_lag_{lag}"] = df["log_return"].shift(lag)

    # ---- Drop leakage ----
    df = df.dropna()

    # ---- Scaling (fit later on train split only) ----
    numeric_features = (
        df
        .select_dtypes(include=["float64", "int64"])
        .columns
        .difference(["next_close"])
    )

    scaler = StandardScaler()
    df[numeric_features] = scaler.fit_transform(df[numeric_features])

    joblib.dump(
        scaler,
        OUT_DIR / f"{ticker}_scaler.pkl"
    )

    # ---- Save ----
    df.to_parquet(
        OUT_DIR / f"{ticker}_features.parquet",
        engine="pyarrow"
    )

    return df


# ---------------------------
# CLI
# ---------------------------
if __name__ == "__main__":
    engineer_features("BTC")

In [5]:
df = pd.read_parquet(OUT_DIR / f"BTC_features.parquet")
print(df.columns.tolist())
print(df.head(1))
print(df.tail(1))

['open', 'high', 'low', 'close', 'volume', 'symbol', 'source', 'building_permits', 'consumer_confidence', 'cpi', 'fed_funds_rate', 'gdp', 'industrial_production', 'money_supply_m1', 'money_supply_m2', 'nonfarm_payrolls', 'pce_inflation', 'ppi', 'retail_sales', 'trade_balance', 'unemployment_rate', 'fed_emb_0', 'fed_emb_1', 'fed_emb_2', 'fed_emb_3', 'fed_emb_4', 'fed_emb_5', 'fed_emb_6', 'fed_emb_7', 'fed_emb_8', 'fed_emb_9', 'fed_emb_10', 'fed_emb_11', 'fed_emb_12', 'fed_emb_13', 'fed_emb_14', 'fed_emb_15', 'fed_emb_16', 'fed_emb_17', 'fed_emb_18', 'fed_emb_19', 'fed_emb_20', 'fed_emb_21', 'fed_emb_22', 'fed_emb_23', 'fed_emb_24', 'fed_emb_25', 'fed_emb_26', 'fed_emb_27', 'fed_emb_28', 'fed_emb_29', 'fed_emb_30', 'fed_emb_31', 'fed_emb_32', 'fed_emb_33', 'fed_emb_34', 'fed_emb_35', 'fed_emb_36', 'fed_emb_37', 'fed_emb_38', 'fed_emb_39', 'fed_emb_40', 'fed_emb_41', 'fed_emb_42', 'fed_emb_43', 'fed_emb_44', 'fed_emb_45', 'fed_emb_46', 'fed_emb_47', 'fed_emb_48', 'fed_emb_49', 'fed_emb_50